In [19]:
import glob
import json
import pandas as pd
import re

In [20]:
def clean(x):
    x = re.sub(r'\w*@\w*', '', x)
    return x

In [22]:
tweets_dict = {}

# Read every *.jsons file in the ./tweet_jsons/ directory and copy them into a pandas dataframe
for filepath in glob.iglob('./tweet_jsons/*.json'):
    tweets_df = pd.read_json(filepath, lines=True)
        
    if not tweets_df.empty:
        tweets_df["content"] = tweets_df["content"].astype(str)
        tweets_df["content"] = tweets_df["content"].apply(clean)
    
    tweets_dict[filepath[14:-5]] = tweets_df
    
print(list(tweets_dict.keys()))

./tweet_jsons/pmarca.json
./tweet_jsons/novogratz.json
./tweet_jsons/rogerkver.json
./tweet_jsons/needacoin.json
./tweet_jsons/crypto_rand.json
./tweet_jsons/ToneVays.json
./tweet_jsons/legendofcrypto.json
./tweet_jsons/CryptoChoe.json
./tweet_jsons/cryptoBully.json
./tweet_jsons/VitalikButerin.json
./tweet_jsons/peterlbrandt.json
./tweet_jsons/NicTrades.json
./tweet_jsons/DiaryofaMademan.json
./tweet_jsons/crypto_God.json
./tweet_jsons/tuurdemeester.json
./tweet_jsons/APompliano.json
./tweet_jsons/lopp.json
./tweet_jsons/nickszabo4.json
./tweet_jsons/IOHK_Charles.json
./tweet_jsons/CryptoCobain.json
./tweet_jsons/6BillionPeople.json
./tweet_jsons/cryptodemedici.json
./tweet_jsons/CryptoOrca.json
./tweet_jsons/VinnyLingham.json
./tweet_jsons/WhalePanda.json
./tweet_jsons/cryptopathic.json
./tweet_jsons/IAMJOSEPHYYOUNG.json
./tweet_jsons/CryptoHustle.json
./tweet_jsons/gavinandresen.json
./tweet_jsons/Crypto_goat_.json
./tweet_jsons/PhilakoneCrypt.json
./tweet_jsons/aantonop.json
./twee

In [23]:
#Keyword filtering: only retain tweets related to cryptocurrency and remove empty lists

keywords = ["btc", "bitcoin", "XBT", "satoshi"]

for key in list(tweets_dict.keys()):
    #print(tweets_dict[key])
    if not tweets_dict[key].empty:
        tweets_dict[key] = tweets_dict[key][tweets_dict[key]["content"].str.contains('|'.join(keywords), na = False, case = False)]
    if tweets_dict[key].empty:
        del tweets_dict[key]

In [24]:
users = list(tweets_dict.keys())
%store users
%store tweets_dict

Stored 'users' (list)
Stored 'tweets_dict' (dict)
